In [30]:
import multiprocessing
import requests
from selenium import webdriver
import lxml.html as ll
import pandas as pd
from selenium.webdriver.chrome.options import Options

import constants as const

In [31]:
def clone(element):
    return ll.etree.fromstring(ll.etree.tostring(element))

In [32]:
def getDriver():
    options = Options()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    return webdriver.Chrome(options=options)

In [41]:
def create(link):
    r = requests.get(link, headers={'User-agent': const.PROFILE_USER_AGENT, 'Cookie': const.PROFILE_COOKIE})

    root_profile = ll.fromstring(r.text)

    string = root_profile.xpath("//span[@class='information season']/a/text()")
    if len(string) == 0:
        string = "Unknown"
    else:
        string = string[0]

    return Comics(float(root_profile.xpath("//div[@class='fl-l score']/div/text()")[0]),
                  int(root_profile.xpath("//span[@class='numbers ranked']/strong/text()")[0][1:]),
                  int(root_profile.xpath("//span[@class='numbers popularity']/strong/text()")[0][1:]),
                  int(root_profile.xpath("//span[@class='numbers members']/strong/text()")[0].replace(',', '')),
                  root_profile.xpath("//span[@class='information type']/a/text()")[0],
                  string,
                  int(root_profile.xpath("//div[@class='recommended']/a/strong/text()")[0]),
                  int(root_profile.xpath("//div[@class='mixed-feelings']/a/strong/text()")[0]),
                  int(root_profile.xpath("//div[@class='not-recommended']/a/strong/text()")[0]))

In [59]:
class Comics:
    def __init__(self, *args):
        self.values = args
        self.current = -1
    
    def __iter__(self):
        return self

    def __next__(self):
        self.current += 1
        if self.current < 9:
            return self.values[self.current]
        raise StopIteration

In [35]:
links = []

amount = {
    'manga': 1500,
    'manhwa': 1000,
    'oneshots': 1000
}

for myType in ['manga', 'manhwa', 'oneshots']:
    for i in range(0, amount[myType], 50):
        driver = getDriver()
        driver.get(const.BASE_URL + myType + "&limit=" + str(i))
        root = ll.fromstring(driver.page_source)
        name_divs = root.xpath("//tbody/tr[@class='ranking-list']")
        for div in name_divs:
            links.append(clone(div).xpath("//a[@class='hoverinfo_trigger fs14 fw-b']")[0].attrib['href'])


In [55]:
data = []
with multiprocessing.Pool(30) as p:
    data.extend(p.map(create, links))


In [56]:
df = pd.DataFrame.from_records(data,
                               columns=['SCORE', 'Ranked', 'Popularity', 'Members', 'Type', 'Magazine', ' Recommended',
                                        'Mixed Feelings', 'Not Recommended'])
df.to_csv('out.csv', index=False)